In [88]:
import itertools
import math
import random
import numpy
from sets import Set

C:\Program Files\Anaconda2\lib\site-packages\ipykernel\__main__.py:5: DeprecationWarning: the sets module is deprecated


In [8]:
def convert(List):
    P = List[:]
    for i in range(len(P)):
        l = P[i]
        newL = []
        for c in l:
            newL.append(int(c))
    #         if c == '0':
    #             newL.append('N')
    #         if c == '1':
    #             newL.append('R')
    #         if c == '2':
    #             newL.append('HR')
        P[i] = newL
    return P

P = ["".join(item) for item in itertools.product("012", repeat=5)]
E =  ["".join(item) for item in itertools.product("012", repeat=5)]

P = convert(P)
E = convert(E)

L = []
for item1 in P:
    for item2 in E:
        pair = []
        pair.append(item1)
        pair.append(item2)
        L.append(pair)

In [9]:
#Precision at K
def precision(res, k):
    counter = 0
    for i in range(k):
        if res[i] != 0:
            counter += 1
    return 1.0*counter / k

def DCG(res, k):
    sum = 0
    for r in range(1,k+1):
        sum += (2**res[r-1] - 1)/math.log(1+r,2)
    return sum

def RBP(res, theta = 0.8):
    sum = 0
    for k in range(1, len(res)):
        sum += res[k-1] * theta**(k-1)*(1 - theta)
    return sum

print L[45]
print precision(L[45][0],k = 3),',', precision(L[45][1],k = 3)
print DCG(L[45][0],k = 3),',',DCG(L[45][1],k = 3)
print DCG([5,5,0,0,1,0,0,0,0,0], 10)
print RBP([0,1,2,1,0])#L[45][1])

[[0, 0, 0, 0, 0], [0, 1, 2, 0, 0]]
0.0 , 0.666666666667
0.0 , 2.13092975357
50.9456751679
0.5184


In [10]:
def measure(pairs, measure, k=5):
    result = []
    for pair in pairs:
        if measure == "precision":
            P = precision(pair[0], k)
            E = precision(pair[1], k)
        
        if measure == "DCG":
            P = DCG(pair[0], k)
            E = DCG(pair[1], k)
            
        if measure == "RBP":
            P = RBP(pair[0])
            E = RBP(pair[1])
         
        diff = E - P
        
        if diff > 0:
            result.append([pair, diff])
            
    return result

res = measure(L, "DCG")
print L[5432]
print res[5432]

print DCG([2, 0, 2, 2, 2], 5) - DCG([0, 0, 2, 2, 1], 5)

[[0, 0, 2, 1, 1], [1, 0, 0, 1, 2]]
[[[0, 0, 2, 2, 1], [2, 0, 2, 2, 2]], 3.7737056144690833]
3.77370561447


In [29]:
#Team Draft Interleaving
def get_clicks(l, number):
    ind_order = range(len(l))
    random.shuffle(ind_order)
    return ind_order[:number]

#a) create the interleaving list
def team_draft_interleave(pair):
#     l1 = zip(pair[0],[0]*len(pair[0]))
#     l2 = zip(pair[1],[1]*len(pair[1]))
    
    lists = [zip(pair[0],[0]*len(pair[0])), zip(pair[1],[1]*len(pair[1]))]
    result = []
    while lists[0] or lists[1]:
        
        if lists[0] and lists[1]:
            selected = random.randint(0,1)
        elif lists[0]:
            selected = 0
        else:
            selected = 1

        doc_index = 0 
        doc = lists[selected][doc_index]
        result.append(doc)
        
        del lists[selected][doc_index]
        other_ind = int(math.fabs(selected-1))
        if lists[other_ind]:
            for i in range(len(lists[other_ind])):
                if lists[other_ind][i][0] == doc[0]:
                    del lists[other_ind][i]
                    break
                 
        temp = selected
        selected = other_ind
        other_ind = selected
        
        if  lists[selected]:
            doc = lists[selected][doc_index]
            result.append(doc)

            del lists[selected][doc_index]
            other_ind = int(math.fabs(selected-1))
            if lists[other_ind]:
                for i in range(len(lists[other_ind])):
                    if lists[other_ind][i][0] == doc[0]:
                        del lists[other_ind][i]
                        break

        
    return result
    
    
#     combined = zip(l1, l2)
#     shuffled = []
#     for line in combined:
#         lst = list(line)
#         random.shuffle(lst)
#         shuffled.append(lst)
#     flattened = [item for sublist in shuffled for item in sublist]
#     return  flattened
    
#b) evaluate the interleaved list
def team_draft_evaluate(interleaved, clicks):
    counters = [0, 0]
    for index in clicks:
        counters[interleaved[index][1]]+=1
    return numpy.sign(counters[0]-counters[1])

def team_draft(pair, n_clicks=2):
    interleaved = team_draft_interleave(pair)
    clicks = get_clicks(interleaved, n_clicks)
    evaluation = team_draft_evaluate(interleaved, clicks)
    return evaluation

team_draft([[0,1,2,3,4,5],[0,1,2,6,4,5]]) #L[1213])

0

In [22]:
#Probabilistic Interleaving

def preprocess(pair):
    c = 0
    new_pair = ([], [])
    for i in range(2):
        for item in pair[i]:
            new_pair[i].append(str(item)+ str(c))
            c+=1
    return new_pair


#a) create the interleaving list
def prob_interleave(pair, tau = 3):
    lists = [zip(pair[0],[0]*len(pair[0])), zip(pair[1],[1]*len(pair[1]))]
    result = []
    while lists[0] or lists[1]:
        
        if lists[0] and lists[1]:
            selected = random.randint(0,1)
        elif lists[0]:
            selected = 0
        else:
            selected = 1
        
        sum = 0
        for i in range(0, len(lists[selected])):
            sum += 1.0/((i+1)**tau)
            
        p = [(1.0/((i+1)**tau))/sum for i in range(0, len(lists[selected]))]
        doc_index = numpy.random.choice(range(len(lists[selected])),1,p=p)
        doc = lists[selected][doc_index]
        result.append(doc)
        del lists[selected][doc_index]
        other_ind = int(math.fabs(selected-1))
        if lists[other_ind]:
            for i in range(len(lists[other_ind])):
                if lists[other_ind][i][0] == doc[0]:
                    del lists[other_ind][i]
                    break
    return result
        
    
#b) evaluate the interleaved list
def prob_evaluate(interleaved, clicks):
    return team_draft_evaluate(interleaved, clicks)

def prob(pair, n_clicks=2):
    interleaved = prob_interleave(pair)
    clicks = get_clicks(interleaved, n_clicks)
    return prob_evaluate(interleaved, clicks)
    
pair = preprocess(L[1234])
# pair = ([1, 2, 3, 4, 5], [2, 3, 4, 5, 6])
prob(pair)

[9, 0, 3, 5, 4, 6, 8, 7, 2, 1]


C:\Program Files\Anaconda2\lib\site-packages\ipykernel\__main__.py:32: VisibleDeprecationWarning: converting an array with ndim > 0 to an index will result in an error in the future
C:\Program Files\Anaconda2\lib\site-packages\ipykernel\__main__.py:34: VisibleDeprecationWarning: converting an array with ndim > 0 to an index will result in an error in the future


0

In [94]:
Q = []
C = []
table = []
with open('YandexRelPredChallenge.txt', 'r') as f:
    for line in f:
        line = line.split()
        table.append(line)
#         if line[2] == 'Q':
#             Q.append(line)
#         else:
#             C.append(line)
f.closed
#print table[-1] 

sum = 0
S = []
for key, group in itertools.groupby(table,lambda x: x[0]):
    queries = []
    for item in group:
        if(item[2]) == 'Q':
            queries.append([])
            queries[-1].append(item[5:])
            queries[-1].append([])
        if(item[2]) == 'C':
            c = item[-1]
            queries[-1][-1].append(c)    
    S.append(queries)
    
print S[-2]





[[['341', '246', '233', '280', '289', '301', '277', '299', '334', '315'], ['341']], [['130920', '125203', '130933', '130916', '130930', '130924', '130929', '130925', '130922', '130926'], []], [['411', '414', '131186', '131189', '131188', '131184', '131185', '131187', '131190', '131191'], ['411']]]


In [97]:
sum1 = 0
sum2 = 0
for s in S:
    clicks = Set([])
    for q in s:
        for c in q[1]:
            clicks.add(c)
    sum1 += len(clicks)
    
    docs = []
    for q in s:
        docs.extend(q[0])
    docs = Set(docs)
    sum2 += len(docs)
    
print 1.0*sum1/sum2

0.143244475768


In [93]:
del sessions